In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx


In [6]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def a_zi_zj(Zi, Zj, W, U, V, W_star):
    return Zi.T @ W @ Zj + Zi.T @ U + V.T @ Zj + W_star

# Parameters and random initializations
N = 100  # Number of nodes
Q = 5    # Number of communities
rho = Q  # Number of communities

# Randomly generate alpha for each community with values between 0.1 and 0.9
alpha = np.random.uniform(0.1, 0.9, Q)

# Generate Zi for each node i based on the new alpha values
Z = np.zeros((N, Q))
for i in range(N):
    for q in range(Q):
        Z[i, q] = np.random.binomial(1, alpha[q])

# Generate the adjacency matrix representing edges
W = np.random.randn(rho, rho)  # W matrix
U = np.random.randn(rho)  # U vector
V = np.random.randn(rho)  # V vector
W_star = np.random.randn()  # W* scalar
X = np.zeros((N, N))  # Initialize the adjacency matrix

for i in range(N):
    for j in range(N):
        if i != j:
            # Calculate edge probability
            prob = sigmoid(a_zi_zj(Z[i], Z[j], W, U, V, W_star))
            # Sample edge existence from Bernoulli distribution
            X[i, j] = np.random.binomial(1, prob)

# Display alpha, first 10 Zi, and first 10x10 submatrix of X for illustration
print("Alpha values for each community:", alpha)
print("First 10 Zi:\n", Z[:10])
print("First 10x10 submatrix of X:\n", X[:10, :10])


Alpha values for each community: [0.53626204 0.30617683 0.40081837 0.1136884  0.63815709]
First 10 Zi:
 [[1. 0. 0. 0. 0.]
 [1. 0. 1. 0. 1.]
 [0. 1. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [1. 0. 1. 0. 1.]
 [1. 0. 1. 0. 0.]
 [1. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 1.]
 [0. 1. 1. 0. 1.]]
First 10x10 submatrix of X:
 [[0. 1. 1. 1. 1. 0. 1. 1. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 1. 1. 1. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 1. 0. 1. 1. 0. 0. 0. 0. 0.]]


Graph.util

In [10]:
import numpy as np
import scipy.stats as stats
import networkx as nx

def osbm_graph(n=100, alphaVect=[1/3, 1/3, 1/3], lambda_=4, epsilon=1, bias=-5.5):
    Q = len(alphaVect)
    Qov = Q + 1
    Z = np.ones((n, Qov))
    W = epsilon * np.ones((Qov, Qov))
    W[:Q, :Q] = -epsilon
    np.fill_diagonal(W, lambda_)
    W[Qov-1, Qov-1] = bias
    nbEmpty = 0
    nbSingle = 0
    nbOverlaps = 0
    
    for q in range(Q):
        Z[:, q] = stats.bernoulli.rvs(alphaVect[q], size=n)
    
    g = nx.DiGraph()
    for i in range(n):
        if sum(Z[i, :-1]) == 0:
            nbEmpty += 1
        elif sum(Z[i, :-1]) == 1:
            nbSingle += 1
        else:
            nbOverlaps += 1
            
        for j in range(n):
            if i != j:
                p = stats.logistic.cdf(np.dot(np.dot(Z[i, :], W), Z[j, :]))
                if stats.bernoulli.rvs(p):
                    g.add_edge(i, j)
    
    print("Total number of vertices:", n)
    print("Number of unclassified vertices:", nbEmpty)
    print("Number of overlaps:", nbOverlaps)
    print("Others:", nbSingle)

    return {'Z': Z[:, :Q], 'graph': g}

def encode_partition(p):
    N = len(p)
    Q = len(np.unique(p))
    Z = np.zeros((N, Q))
    for q in range(Q):
        Z[p == q, q] = 1
    return Z

def euclidean_dist(x, y):
    return np.dot(x - y, x - y)

def get_connected_nodes(x):
    if x.shape[0] == x.shape[1]:
        nbrNodes = x.shape[0]
        not_connected = np.where((np.sum(x, axis=0) + np.sum(x, axis=1)) == 0)[0]
        connected_nodes = np.setdiff1d(np.arange(nbrNodes), not_connected)
    return connected_nodes

def getModel_osbm(object, **kwargs):
    if not isinstance(object, dict) or 'Z' not in object or 'graph' not in object:
        raise ValueError("Not a osbm object")
    
    q = kwargs.get('q', None)
    
    # Placeholder for additional logic related to 'q'
    
    res = {'q': q, 'Z': object['Z']}
    return res

def edges_to_adj_mat(x, symmetrize=False, n=None):
    if n is None:
        n = np.max(x)
    mat = np.zeros((n, n))
    mat[x[0, :], x[1, :]] = 1
    if symmetrize:
        mat[x[1, :], x[0, :]] = 1
    return mat

def adj_mat_to_edges(x, directed=False):
    if x.shape[0] == x.shape[1]:
        nbrNodes = x.shape[0]
        not_connected = np.where((np.sum(x, axis=0) + np.sum(x, axis=1)) == 0)[0]
        if len(not_connected) > 0:
            x = np.delete(np.delete(x, not_connected, axis=0), not_connected, axis=1)
            print("Some nodes are not connected to the network")
        if directed:
            m = np.transpose(np.where(np.transpose(x) == 1))
        else:
            m = np.transpose(np.where(np.triu(x) == 1))
    return m


In [13]:
import networkx as nx

# 生成图g
result = osbm_graph()
g = result['graph']  # 从结果中提取图结构

# 查看图信息
print("Number of nodes:", g.number_of_nodes())
print("Number of edges:", g.number_of_edges())


Total number of vertices: 100
Number of unclassified vertices: 28
Number of overlaps: 16
Others: 56
Number of nodes: 97
Number of edges: 1723


init

In [ ]:
.onLoad = function(libname, pkgname) {
  console.log("\n");
  console.log("      OSBM package\n");
}


spmgraph

In [ ]:
class spmgraph {
  constructor() {
    this.name = 'Unnamed';
    this.edgesrcs = [];
    this.edgedests = [];
    this.nodenames = [];
    this.nodeindexes = [];
  }

  addedge(dest, srcindex, initenv) {
    let g = initenv.instance;
    let destindex = g.nodenames.indexOf(dest);
    if (destindex === -1) {
      destindex = g.nodenames.length;
      g.nodenames[destindex] = dest;
      g.nodeindexes[dest] = destindex;
    }
    g.edgesrcs.push(srcindex);
    g.edgedests.push(destindex);
    initenv.instance = g;
  }

  parseline(line, initenv) {
    let fields = line.split(/\s+/);
    if (fields.length > 1) {
      let src = fields[0];
      let dest = fields.slice(1);
      let g = initenv.instance;
      let nextindex = g.nodeindexes.length;
      let srcindex = g.nodenames.indexOf(src);
      if (srcindex === -1) {
        srcindex = g.nodenames.length;
        g.nodenames[srcindex] = src;
        g.nodeindexes[src] = srcindex;
      }
      initenv.instance = g;
      dest.forEach((d) => this.addedge(d, srcindex, initenv));
    }
  }

  readfromfile(filename, initenv) {
    let lines = filename.split('\n');
    lines.forEach((line) => this.parseline(line, initenv));
    return true;
  }

  addadjmatedge(index, initenv, nnodes) {
    index = index - 1;
    let g = initenv.instance;
    let col = Math.floor(index / nnodes);
    let row = index - col * nnodes + 1;
    col = col + 1;
    let srcindex = g.nodenames.indexOf(row);
    if (srcindex === -1) {
      srcindex = g.nodenames.length;
      g.nodenames[srcindex] = row;
      g.nodeindexes[row] = srcindex;
    }
    let destindex = g.nodenames.indexOf(col);
    if (destindex === -1) {
      destindex = g.nodenames.length;
      g.nodenames[destindex] = col;
      g.nodeindexes[col] = destindex;
    }
    g.edgesrcs.push(srcindex);
    g.edgedests.push(destindex);
    initenv.instance = g;
  }

  buildfromadjacencymatrix(adjmat, initenv) {
    let nonnulls = [];
    adjmat.forEach((row, i) => {
      row.forEach((val, j) => {
        if (val !== 0) {
          nonnulls.push(i * adjmat.length + j);
        }
      });
    });
    let nnodes = adjmat.length;
    nonnulls.forEach((index) => this.addadjmatedge(index, initenv, nnodes));
    return true;
  }
}

function initialize_spmgraph(Object, initialdata = null) {
  let initialized = false;
  if (typeof initialdata === 'string') {
    Object.name = initialdata;
    let initenv = {};
    initenv.instance = Object;
    initialized = Object.readfromfile(initialdata, initenv);
    Object = initenv.instance;
  }
  if (Array.isArray(initialdata)) {
    let initenv = {};
    initenv.instance = Object;
    initialized = Object.buildfromadjacencymatrix(initialdata, initenv);
    Object = initenv.instance;
  }
  if (initialized === false) {
    console.warn('Could not correctly initialize spmgraph object.');
  }
  return Object;
}

function getEdges(g, withlabels = false) {
  let edgelist = [...g.edgesrcs, ...g.edgedests];
  if (withlabels === true) {
    edgelist = edgelist.map((index) => g.nodenames[index]);
  }
  let result = [];
  for (let i = 0; i < edgelist.length; i += 2) {
    result.push([edgelist[i], edgelist[i + 1]]);
  }
  return result;
}

function indextolabel(index, g) {
  return g.nodenames[index];
}

function getAdjacencyMatrix(g, withlabels = false) {
  let nnodes = g.nodenames.length;
  let adjmat = Array.from({ length: nnodes }, () => Array(nnodes).fill(0));
  for (let i = 0; i < g.edgesrcs.length; i++) {
    let src = g.edgesrcs[i];
    let dest = g.edgedests[i];
    adjmat[src][dest] = 1;
    adjmat[dest][src] = 1;
  }
  if (withlabels === true) {
    let nodenames = g.nodenames.map((index) => indextolabel(index, g));
    adjmat = adjmat.map((row, i) => {
      row.unshift(nodenames[i]);
      return row;
    });
    nodenames.unshift('');
    adjmat.unshift(nodenames);
  }
  return adjmat;
}

class spmgraph {
  constructor() {
    this.name = 'Unnamed';
    this.edgesrcs = [];
    this.edgedests = [];
    this.nodenames = [];
    this.nodeindexes = [];
  }

  addedge(dest, srcindex, initenv) {
    let g = initenv.instance;
    let destindex = g.nodenames.indexOf(dest);
    if (destindex === -1) {
      destindex = g.nodenames.length;
      g.nodenames[destindex] = dest;
      g.nodeindexes[dest] = destindex;
    }
    g.edgesrcs.push(srcindex);
    g.edgedests.push(destindex);
    initenv.instance = g;
  }

  parseline(line, initenv) {
    let fields = line.split(/\s+/);
    if (fields.length > 1) {
      let src = fields[0];
      let dest = fields.slice(1);
      let g = initenv.instance;
      let nextindex = g.nodeindexes.length;
      let srcindex = g.nodenames.indexOf(src);
      if (srcindex === -1) {
        srcindex = g.nodenames.length;
        g.nodenames[srcindex] = src;
        g.nodeindexes[src] = srcindex;
      }
      initenv.instance = g;
      dest.forEach((d) => this.addedge(d, srcindex, initenv));
    }
  }

  readfromfile(filename, initenv) {
    let lines = filename.split('\n');
    lines.forEach((line) => this.parseline(line, initenv));
    return true;
  }

  addadjmatedge(index, initenv, nnodes) {
    index = index - 1;
    let g = initenv.instance;
    let col = Math.floor(index / nnodes);
    let row = index - col * nnodes + 1;
    col = col + 1;
    let srcindex = g.nodenames.indexOf(row);
    if (srcindex === -1) {
      srcindex = g.nodenames.length;
      g.nodenames[srcindex] = row;
      g.nodeindexes[row] = srcindex;
    }
    let destindex = g.nodenames.indexOf(col);
    if (destindex === -1) {
      destindex = g.nodenames.length;
      g.nodenames[destindex] = col;
      g.nodeindexes[col] = destindex;
    }
    g.edgesrcs.push(srcindex);
    g.edgedests.push(destindex);
    initenv.instance = g;
  }

  buildfromadjacencymatrix(adjmat, initenv) {
    let nonnulls = [];
    adjmat.forEach((row, i) => {
      row.forEach((val, j) => {
        if (val !== 0) {
          nonnulls.push(i * adjmat.length + j);
        }
      });
    });
    let nnodes = adjmat.length;
    nonnulls.forEach((index) => this.addadjmatedge(index, initenv, nnodes));
    return true;
  }
}

function initialize_spmgraph(Object, initialdata = null) {
  let initialized = false;
  if (typeof initialdata === 'string') {
    Object.name = initialdata;
    let initenv = {};
    initenv.instance = Object;
    initialized = Object.readfromfile(initialdata, initenv);
    Object = initenv.instance;
  }
  if (Array.isArray(initialdata)) {
    let initenv = {};
    initenv.instance = Object;
    initialized = Object.buildfromadjacencymatrix(initialdata, initenv);
    Object = initenv.instance;
  }
  if (initialized === false) {
    console.warn('Could not correctly initialize spmgraph object.');
  }
  return Object;
}

function getEdges(g, withlabels = false) {
  let edgelist = [...g.edgesrcs, ...g.edgedests];
  if (withlabels === true) {
    edgelist = edgelist.map((index) => g.nodenames[index]);
  }
  let result = [];
  for (let i = 0; i < edgelist.length; i += 2) {
    result.push([edgelist[i], edgelist[i + 1]]);
  }
  return result;
}

function indextolabel(index, g) {
  return g.nodenames[index];
}

function getAdjacencyMatrix(g, withlabels = false) {
  let nnodes = g.nodenames.length;
  let adjmat = Array.from({ length: nnodes }, () => Array(nnodes).fill(0));
  for (let i = 0; i < g.edgesrcs.length; i++) {
    let src = g.edgesrcs[i];
    let dest = g.edgedests[i];
    adjmat[src][dest] = 1;
    adjmat[dest][src] = 1;
  }
  if (withlabels === true) {
    let nodenames = g.nodenames.map((index) => indextolabel(index, g));
    adjmat = adjmat.map((row, i) => {
      row.unshift(nodenames[i]);
      return row;
    });
    nodenames.unshift('');
    adjmat.unshift(nodenames);
  }
  return adjmat;
}

module.exports = {
  spmgraph,
  initialize_spmgraph,
  getEdges,
  getAdjacencyMatrix,
};




Gplot

Gplot-internals

obsm

In [ ]:
import numpy as np

def class_ind(cl):
    n = len(cl)
    cl = np.array(cl)
    x = np.zeros((n, len(np.unique(cl))))
    x[np.arange(n) + n * (cl - 1)] = 1
    x = x.astype(int)
    return x

def osbm(X, qmin=2, qmax=None, nbiter=50, fpnbiter=5, nstart=1, init=None):
    epsoutermin = 1e-8
    epsinnermin = 1e-4
    directed = True
    
    if isinstance(X, str):
        g = spmgraph(X)
        X = Edges2AdjMat(getEdges(g))
        ConnectedNodes = np.array(g.nodeindexes, dtype=int)
    elif X.shape[0] == X.shape[1]:
        ConnectedNodes = getConnectedNodes(X, directed=directed)
    else:
        ConnectedNodes = np.arange(1, np.max(X)+1)
        X = Edges2AdjMat(X)
    
    N = X.shape[0]
    A = X
    X = X - 1/2
    np.fill_diagonal(X, 0)
    
    if qmax is None:
        qmax = qmin
    
    y = []
    youter = []
    criterion = np.zeros((nstart, qmax-qmin+1))
    
    for run in range(nstart):
        testTrash = 1
        yinner = []
        
        for q in range(qmin, qmax+1):
            qov = q + 1
            eta0 = np.repeat(1/2, q)
            zeta0 = np.repeat(1/2, q)
            eta = eta0
            zeta = zeta0
            an = 1
            bn = 1
            
            S = np.zeros((qov**2, qov**2))
            vecW = np.random.normal(size=qov**2)
            xi = np.full((N, N), 0.001)
            np.fill_diagonal(xi, 0)
            
            if testTrash == 1:
                kZ = class_ind(init[q-qmin])
                
                if testTrash == 0:
                    tau0 = kZ
                else:
                    tau0 = np.zeros((N, qov))
                    tau0[:, :-1] = kZ
                
                tau0[tau0 < np.finfo(float).tiny] = np.finfo(float).tiny
                tau0[tau0 > 1/(1+np.finfo(float).eps)] = 1/(1+np.finfo(float).eps)
                tau0[:, -1] = 1
                
                nite = 0
                l = np.zeros(nbiter)
                
                res = bosbm_R(N, q, eta0, zeta0, eta, zeta, an, bn, X, xi, S, vecW, tau0, nbiter, fpnbiter, epsoutermin, epsinnermin, l, nite)
            
            nite = res['nite']
            l = res['l'][:nite]
            tau = res['tau'].reshape((N, qov))
            
            Z = np.round(tau)
            yinner.append({'W': res['W'].reshape((qov, qov)),
                           'S': res['S'].reshape((qov**2, qov**2)),
                           'alphas': res['eta'] / (res['eta'] + res['zeta']),
                           'Taus': tau[:, :-1].T,
                           'Xi': res['xi'].reshape((N, N)),
                           'an': res['an'],
                           'bn': res['bn'],
                           'l': l,
                           'eta': res['eta'],
                           'zeta': res['zeta'],
                           'criterion': np.max(l)})
            
            criterion[run, q-qmin] = yinner[q-qmin]['criterion']
        
        youter.append(yinner)
    
    i_opt = np.argmax(criterion, axis=0)
    i_res = 0
    y = []
    
    for q in range(qmin, qmax+1):
        ytemp = youter[i_opt[i_res]][i_res]
        y.append({'criterion': ytemp['criterion'],
                  'W': ytemp['W'],
                  'S': ytemp['S'],
                  'alphas': ytemp['alphas'],
                  'Taus': ytemp['Taus'],
                  'Xi': ytemp['Xi'],
                  'an': ytemp['an'],
                  'bn': ytemp['bn'],
                  'l': ytemp['l'],
                  'eta': ytemp['eta'],
                  'zeta': ytemp['zeta']})
        i_res += 1
    
    result = {'method': 'variational bayes',
              'nnodes': N,
              'map': ConnectedNodes,
              'edges': AdjMat2Edges(A, directed=True),
              'qmin': qmin,
              'qmax': qmax,
              'output': y,
              'directed': True}
    
    return result

def is_osbm(x):
    return isinstance(x, dict) and x.get('method') == 'variational bayes'

def is_osbm_graph(x):
    return isinstance(x, dict) and x.get('method') == 'variational bayes'

def get_model_osbm(object, q=None):
    if not is_osbm(object):
        raise ValueError("Not an osbm object")
    
    x = object
    
    if q is not None:
        if q not in range(x['qmin'], x['qmax']+1):
            raise ValueError("Bad value of 'q'")
    
    if q is None:
        crit = [output['criterion'] for output in x['output']]
        i = np.argmax(crit)
        q = len(x['output'][i]['alphas'])
    
    i = q - x['qmin']
    res = {'q': q,
           'criterion': x['output'][i]['criterion'],
           'W': x['output'][i]['W'],
           'S': x['output'][i]['S'],
           'alphas': x['output'][i]['alphas'],
           'Taus': x['output'][i]['Taus'],
           'Xi': x['output'][i]['Xi']}
    
    return res

def plot_criterion(x, q):
    title = "Bayesian criterion vs class number"
    y_lab = "Bayesian criterion"
    
    Q = [len(output['alphas']) for output in x['output']]
    crit = [output['criterion'] for output in x['output']]
    
    plt.plot(Q, crit)
    plt.xlabel("Number of classes")
    plt.ylabel(y_lab)
    plt.title(title)
    plt.axvline(x=q, color="red", linestyle="--")
    plt.show()

def plot_osbm_graph(x, pos=None):
    Gplot(x['x'], type="pie.classes", node.pie.coef=x['Z'].T, main="", directed=True, coord=pos)

def plot_osbm(x, q=None, frame=[1, 2, 3, 4]):
    if is_osbm_graph(x):
        plot_osbm_graph(x)
    elif not is_osbm(x):
        raise ValueError("Not an osbm object")
    
    res = x
    
    if q is not None:
        if q not in range(res['qmin'], res['qmax']+1):
            raise ValueError("Bad value of 'q'")
    
    if not all(f in range(1, 6) for f in frame):
        raise ValueError("Bad frame number")
    
    index = [i for i, f in enumerate(frame) if f > 5]
    frame = [f for f in frame if f <= 5]
    nb_frame = len(frame)
    
    if nb_frame > 4:
        frame = frame[:4]
        nb_frame = 4
    
    n = res['x'].shape[0]
    
    if q is None:
        crit = [output['criterion'] for output in res['output']]
        i = np.argmax(crit)
        q = len(res['output'][i]['alphas'])
    
    cluster = np.argmax(res['output'][q-res['qmin']]['Taus'], axis=1)
    alphas = res['output'][q-res['qmin']]['alphas']
    
    nb_col = 2
    nb_lin = 2
    
    if nb_frame == 1:
        nb_col = 1
        nb_lin = 1
    elif nb_frame == 2:
        nb_col = 2
        nb_lin = 1
    
    plt.figure(figsize=(12, 8))
    
    if 1 in frame:
        plot_criterion(res, q)
    
    if 2 in frame:
        pass
    
    if 3 in frame:
        pass
    
    if 4 in frame:
        mat = np.zeros((res['nnodes'], res['nnodes']))
        mat[res['map'][res['edges'][1]], res['map'][res['edges'][0]]] = 1
        Z = get_model_osbm(res)
        Taus = Z['Taus'].T
        Taus[Taus < 10e-4] = 0
        Gplot(mat, type="pie.classes", node.pie.coef=Taus, main="Overlapping graph", directed=res['directed'])
    
    if 5 in frame:
        mat = np.zeros((res['nnodes'], res['nnodes']))
        mat[res['map'][res['edges'][1]], res['map'][res['edges'][0]]] = 1
        Z = get_model_osbm(res)
        Taus = Z['Taus'].T
        threshold = 0.7
        Taus[Taus < threshold] = 0
        Taus[Taus >= threshold] = 1
        Gplot(mat, type="pie.classes", node.pie.coef=Taus, main="", directed=res['directed'])
    
    plt.tight_layout()
    plt.show()


